In [1]:
#Import Dependencies
import pandas as pd
from datetime import date
from pathlib import Path

In [2]:
#Set Paths for Data CSV's
clinp=Path("Resources/WHO_NREVSS_Clinical_Labs.csv")
pubp=Path("Resources/WHO_NREVSS_Public_Health_Labs.csv")
comp=Path("Resources/WHO_NREVSS_Combined_prior_to_2015_16.csv")

In [90]:
#Read Data from the CSV's
clinical = pd.read_csv(clinp,header=1)
public = pd.read_csv(pubp, header=1)
combo2015= pd.read_csv(comp, header=1)

public.head()

,REGION TYPE,REGION,SEASON_DESCRIPTION,TOTAL SPECIMENS,A (2009 H1N1),A (H3),A (Subtyping not Performed),B,BVic,BYam,H3N2v,A (H5)
0,States,Alabama,Season 2015-16,256,59,16,1,2,2,2,0,0
1,States,Alaska,Season 2015-16,4691,607,98,0,231,1,2,0,0
2,States,Arizona,Season 2015-16,2110,762,580,0,13,58,399,0,0
3,States,Arkansas,Season 2015-16,128,20,8,0,1,12,0,0,0
4,States,California,Season 2015-16,12820,1462,854,35,775,309,803,0,0


In [91]:
#Correct the 2014 week 53 which must have been a recording error, as 2014 only had 52 weeks per the ISO calendar
combo2015.loc[(combo2015["WEEK"]==53),"WEEK"]=52

#Isolate the data taken during flu season for weekly data (October-May)
clinical["Season"]=clinical[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
clinical=clinical.drop(clinical[(clinical.Season<10)&(clinical.Season>5)].index)

combo2015["Season"]=combo2015[["YEAR","WEEK"]].apply(lambda row: date.fromisocalendar(int(row['YEAR']), int(row['WEEK']),1).month,axis=1)
combo2015=combo2015.drop(combo2015[(combo2015.Season<10)&(combo2015.Season>5)].index)

In [92]:
#Drop Null (X only) entries
clinical=clinical[clinical["TOTAL SPECIMENS"]!="X"]
public=public[public["TOTAL SPECIMENS"]!="X"]
combo2015=combo2015[combo2015["TOTAL SPECIMENS"]!="X"]


#Replace isolated X's with 0's
for coln in [5,6]:
    col=clinical.columns[coln]
    clinical.loc[clinical[col]=="X",col]=0

for coln in range(4,12):
    col=public.columns[coln]
    public.loc[public[col]=="X",col]=0

for coln in range(6,14):
    col=combo2015.columns[coln]
    combo2015.loc[combo2015[col]=="X",col]=0


In [93]:
#Convert date data into "Season year1-year2" format present in the Public dataset
clinical.loc[(clinical["Season"]<10),"YEAR"]=clinical["YEAR"].apply(lambda x: x-1)
clinical["YEAR_2"]=clinical["YEAR"].apply(lambda x: str(x+1)[2:4])
clinical["Season"]=clinical[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)

combo2015.loc[(combo2015["Season"]<10),"YEAR"]=combo2015["YEAR"].apply(lambda x: x-1)
combo2015["YEAR_2"]=combo2015["YEAR"].apply(lambda x: str(x+1)[2:4])
combo2015["Season"]=combo2015[["YEAR","YEAR_2"]].apply(lambda row: "Season "+str(row["YEAR"])+"-"+row["YEAR_2"],axis=1)

#Convert Numerical Data into integers
clinical=clinical.astype({"TOTAL SPECIMENS":"int32","TOTAL A":"int32","TOTAL B":"int32"})
combo2015=combo2015.astype({"TOTAL SPECIMENS":"int32","A (2009 H1N1)":"int32","A (H1)":"int32","A (H3)":"int32",
                            "A (Subtyping not Performed)":"int32","A (Unable to Subtype)":"int32","B":"int32",
                            "H3N2v":"int32","A (H5)":"int32"})
public=public.astype({"TOTAL SPECIMENS":"int32","A (2009 H1N1)":"int32","A (H3)":"int32",
                            "A (Subtyping not Performed)":"int32","B":"int32","BVic":"int32","BYam":"int32",
                            "H3N2v":"int32","A (H5)":"int32"})

public.head()

#Groupby Season and Region, summing the weekly data into season data
#clinical=clinical[["Season","REGION","TOTAL SPECIMENS","TOTAL A","TOTAL B"]].groupby(["Season","REGION"]).sum(numeric_only=1)
#print(clinical)

,REGION TYPE,REGION,SEASON_DESCRIPTION,TOTAL SPECIMENS,A (2009 H1N1),A (H3),A (Subtyping not Performed),B,BVic,BYam,H3N2v,A (H5)
0,States,Alabama,Season 2015-16,256,59,16,1,2,2,2,0,0
1,States,Alaska,Season 2015-16,4691,607,98,0,231,1,2,0,0
2,States,Arizona,Season 2015-16,2110,762,580,0,13,58,399,0,0
3,States,Arkansas,Season 2015-16,128,20,8,0,1,12,0,0,0
4,States,California,Season 2015-16,12820,1462,854,35,775,309,803,0,0
